In [1]:
!pip install -e git+https://github.com/PasaOpasen/geneticalgorithm2.git#egg=geneticalgorithm2
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining geneticalgorithm2 from git+https://github.com/PasaOpasen/geneticalgorithm2.git#egg=geneticalgorithm2
  Cloning https://github.com/PasaOpasen/geneticalgorithm2.git to ./src/geneticalgorithm2
  Running command git clone -q https://github.com/PasaOpasen/geneticalgorithm2.git /content/src/geneticalgorithm2
     |████████████████████████████████| 44 kB 1.6 MB/s 
  Created wheel for func-timeout: filename=func_timeout-4.3.5-py3-none-any.whl size=15096 sha256=b100344b5d9997d4897da8d1b1665fd27b3893ed1583065f18d85787ecea0634
  Stored in directory: /root/.cache/pip/wheels/a8/92/ca/5bbab358275e310af23b73fc32ebf37d6a7a08c87c8d2cdbc1
Successfully built func-timeout
  Running setup.py develop for geneticalgorithm2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.2 MB/s 


In [2]:
from os import listdir
import matplotlib.pyplot as plt
from matplotlib import image as pltimage
import numpy as np
import pandas as pd
import cv2, random, shutil, math, os, sys, multiprocessing
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from skimage import img_as_float
from skimage.metrics import peak_signal_noise_ratio
from skimage.filters import unsharp_mask
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, expon, randint
from keras.layers import Input, Lambda, Dense, Flatten, concatenate, LeakyReLU, PReLU, Dropout
#from keras.layers.core import Dropout
from keras.models import Model, Sequential, clone_model
#from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications import vgg16, resnet_v2, densenet, inception_resnet_v2#, VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, RepeatedStratifiedKFold, cross_val_score
from sklearn import preprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
from keras import backend as K
from keras.utils import Sequence
import tensorflow as tf 
import tensorflow_addons as tfa
import copy
import os
import torch
import torchvision

In [4]:
import pickle
def save_object(obj, filename):
    """Saved python object as pickle file to current directory

    Parameters
    ----------
    obj : any
        The python object to save
    filename : str
        The file name to use for .pkl file (format: '_.pkl')

    Returns
    -------
    None
    """
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
        
def read_object(name):
    """Read pickle file as python object from current directory

    Parameters
    ----------
    name : str
        The .pkl file name excluding '.pkl' extension (format: name.pkl)

    Returns
    -------
    any
        The python object contained in the pickle file
    """
    fName = str(name + '.pkl')
    with open(fName, 'rb') as f:  # with statement avoids file leak
        return pickle.load(f)

In [5]:
# load all images in a directory
loaded_images = {'0': list(), '1': list(), '2': list()}
min_dim0 = 500
min_dim1 = 500
#dir = '/content/drive/MyDrive/Ultrasound-Mice/'
dir = 'data/'
mouse_ids = {'0': list(), '1': list(), '2': list()}
mouse_ids_unique = {'0': list(), '1': list(), '2': list()}
filenames = {'0': list(), '1': list(), '2': list()}

for stage in range(3):
  for filename in listdir(dir + str(stage)):
      mouse_ids[str(stage)].append(filename.split('_')[0])
      # load image
      img_data = pltimage.imread(dir + str(stage) + '/' + filename)
      # store loaded image
      loaded_images[str(stage)].append(img_data)
      min_dim0 = np.min([min_dim0, img_data.shape[0]])
      min_dim1 = np.min([min_dim1, img_data.shape[1]])
      print('> loaded %s %s' % (filename, img_data.shape))
  mouse_ids_unique[str(stage)] = np.unique(mouse_ids[str(stage)]).tolist()

print(min_dim0, min_dim1, str(3))

> loaded R145849_D7_fr_26.jpg (181, 329, 3)
> loaded R145849_D7_fr_22.jpg (174, 316, 3)
> loaded R145848_D7_fr_74.jpg (168, 304, 3)
> loaded RR145848_D7_fr_53.jpg (202, 348, 3)
> loaded RR145848_D7_fr_47.jpg (198, 340, 3)
> loaded R145848_D7_fr_53.jpg (183, 294, 3)
> loaded R145848_D7_fr_50.jpg (183, 294, 3)
> loaded R145848_D7_fr_71.jpg (168, 304, 3)
> loaded R145849_D7_fr_21.jpg (168, 304, 3)
> loaded RR145848_D7_fr_45.jpg (198, 340, 3)
> loaded R145849_D7_fr_20.jpg (168, 304, 3)
> loaded RR145848_D7_fr_52.jpg (198, 340, 3)
> loaded R145849_D7_fr_30.jpg (181, 329, 3)
> loaded RR145848_D7_fr_55.jpg (198, 364, 3)
> loaded R145848_D7_fr_73.jpg (168, 304, 3)
> loaded RR145848_D7_fr_49.jpg (198, 340, 3)
> loaded R145848_D7_fr_70.jpg (189, 304, 3)
> loaded RR145848_D7_fr_48.jpg (198, 340, 3)
> loaded R145849_D7_fr_23.jpg (170, 309, 3)
> loaded R145849_D7_fr_25.jpg (181, 329, 3)
> loaded R145848_D7_fr_72.jpg (168, 304, 3)
> loaded R145849_D7_fr_24.jpg (181, 329, 3)
> loaded RR145848_D7_fr_4

In [6]:
mouse_ids_te_unique = {'0': list(), '1': list(), '2': list()}
mouse_ids_tr_unique = {'0': list(), '1': list(), '2': list()}
mouse_ids_te = {'0': list(), '1': list(), '2': list()}
mouse_ids_tr = {'0': list(), '1': list(), '2': list()}
img_te = {'0': list(), '1': list(), '2': list()}
img_tr = {'0': list(), '1': list(), '2': list()}
img_tr_denoised = {'0': list(), '1': list(), '2': list()}
psnrs1 = {'0': list(), '1': list(), '2': list()}
random.seed(123)

for stage in range(3):
    mouse_ids_te_unique[str(stage)] = random.sample(mouse_ids_unique[str(stage)], len(mouse_ids_unique[str(stage)])//3)
    mouse_ids_tr_unique[str(stage)] = list(set(mouse_ids_unique[str(stage)]) ^ set(mouse_ids_te_unique[str(stage)]))
    for filename in listdir(dir + str(stage)):
        # load image
        img_data = pltimage.imread(dir + str(stage) + '/' + filename)
        if filename.startswith(tuple(mouse_ids_te_unique[str(stage)])):
            mouse_ids_te[str(stage)].append(filename.split('_')[0])
            # store loaded image
            img_te[str(stage)].append(img_as_float(cv2.resize(img_data, (min_dim1, min_dim0))))
            #shutil.copy(dir + str(stage) +'/' + filename, dir + 'test/' + str(stage))
        else:
            mouse_ids_tr[str(stage)].append(filename.split('_')[0])
            unsharp_img = unsharp_mask(img_as_float(img_data), multichannel = True)
            denoised_img = denoise_wavelet(img_as_float(unsharp_img),
                                          multichannel = True, convert2ycbcr=True,
                                          method='BayesShrink', mode='soft', rescale_sigma=True)
            resized_img = cv2.resize(denoised_img, (min_dim1, min_dim0))
            img_tr[str(stage)].append(img_as_float(cv2.resize(img_data, (min_dim1, min_dim0))))
            img_tr_denoised[str(stage)].append(resized_img)
            #shutil.copy(dir + str(stage) +'/' + filename, dir + 'train/' + str(stage))
            #plt.imsave(dir + 'train_preprocessed/' + str(stage) + '/' + filename, denoised_img)

In [7]:
tr = pd.DataFrame({'Image': [img for stage in range(3) for img in img_tr[str(stage)]],
                   'Mouse_ID': [id for stage in range(3) for id in mouse_ids_tr[str(stage)]],
                   'Stage': [stage for stage in range(3) for id in mouse_ids_tr[str(stage)]]
                    })

te = pd.DataFrame({'Image': [img for stage in range(3) for img in img_te[str(stage)]],
                   'Mouse_ID': [id for stage in range(3) for id in mouse_ids_te[str(stage)]],
                   'Stage': [stage for stage in range(3) for id in mouse_ids_te[str(stage)]]
                    })

In [8]:
def build_cnn_base(i, kind = "vgg", nlayer = 2, loss = 'categorical_crossentropy', metrics = ['accuracy'],
                     l2_penalty = 0.1, optimizer = 'nadam', optimizer2 = 'nadam',
                     input_shape = (244, 244, 3), n_class = 3, random_state = 123, end = True, start = False):
    np.random.seed(random_state)
    tf.keras.utils.set_random_seed(random_state)
    if(kind == "vgg"): 
        cnn = vgg16.VGG16(input_shape = input_shape, weights='imagenet', include_top=False)
    elif(kind == "resnet"):
        cnn = resnet_v2.ResNet50V2(input_shape = input_shape, weights='imagenet', include_top=False)
    elif(kind == "densenet"):
        cnn = densenet.DenseNet121(input_shape = input_shape, weights='imagenet', include_top=False)
    elif(kind == "inception"):
        cnn = inception_resnet_v2.InceptionResNetV2(input_shape = input_shape, weights='imagenet', include_top=False)

    x = Flatten()(cnn.output)
    y = Dense(n_class, activation='softmax')(x)
    model = Model(inputs = cnn.input, outputs = y)
    count = 0
    for layer in model.layers:
      count +=1
      layer.trainable = False
      layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
    if(end):
      for layer in model.layers[count-nlayer-2:]:
        layer.trainable = True
    if(start):
      for layer in model.layers[0:nlayer]:
        layer.trainable = True
    opt = [(optimizer, model.layers[0:-1]), (optimizer2, model.layers[-1])]
    opt = tfa.optimizers.MultiOptimizer(opt)
    model.compile(loss=loss, optimizer=opt, metrics=metrics)
    return(model)

In [9]:
scv = StratifiedKFold(n_splits = 3, random_state = 123, shuffle = True)
imgs = np.array([img for img in tr.Image.values])
y = tr.Stage.values
id_df = tr.iloc[:,1:]
id_df_unique = id_df.drop_duplicates('Mouse_ID')
train_idx, val_idx = next(scv.split(id_df_unique.loc[:,'Mouse_ID'], id_df_unique.Stage))
X_tr = imgs[id_df.loc[:,'Mouse_ID'].isin(id_df_unique.iloc[train_idx,0])]
y_tr = y[id_df.loc[:,'Mouse_ID'].isin(id_df_unique.iloc[train_idx,0])]
X_val = imgs[id_df.loc[:,'Mouse_ID'].isin(id_df_unique.iloc[val_idx,0])]
y_val = y[id_df.loc[:,'Mouse_ID'].isin(id_df_unique.iloc[val_idx,0])]

In [10]:
def make_FCNet(cnns, activation, n_output, loss, optimizer, clipvalue, metrics,
               nhidden1, nhidden2, nhidden3, dropout1, dropout2, dropout3, lr, seed):
    tf.random.set_seed(seed)
    tf.keras.utils.set_random_seed(seed)
    for cnn in cnns:
        for layer in cnn.layers:
            layer.trainable = False
    ensemble_visible = [cnn.input for cnn in cnns]
    ensemble_outputs = [cnn.layers[-2].output for cnn in cnns]
    merge = concatenate(ensemble_outputs)

    hidden1 = Dense(nhidden1, activation=activation, kernel_regularizer = l2(0.2))(merge)#
    drop1 = Dropout(dropout1, seed = seed)(hidden1)
    hidden2 = Dense(nhidden2, activation=activation, kernel_regularizer = l2(0.2))(drop1)#, kernel_regularizer = l2(0.2)
    drop2 = Dropout(dropout2, seed = seed)(hidden2)
    hidden3 = Dense(nhidden3, activation=activation, kernel_regularizer = l2(0.2))(drop2)#, kernel_regularizer = l2(0.2)
    drop3 = Dropout(dropout2, seed = seed)(hidden3)
    output = Dense(n_output, activation='softmax')(drop3)
    cnn_ensemble = Model(inputs=ensemble_visible, outputs=output)
    cnn_ensemble.compile(loss=loss, 
                         optimizer=optimizers.Nadam(#clipvalue = clipvalue, 
                                                    learning_rate=lr) if optimizer == 'nadam' else optimizers.Adamax(#clipvalue = clipvalue, 
                                                                                                                     learning_rate=lr), 
                         metrics=metrics)
    #K.set_value(cnn_ensemble.optimizer.learning_rate, lr)
    return(cnn_ensemble)

In [11]:
def ids_to_samples(cv, X, y, id_df, id_col_name = 'Mouse_ID'):
      sample_ids = list()
      id_df_unique = id_df.drop_duplicates(id_col_name)
      for train_idx, val_idx in cv.split(id_df_unique.loc[:,id_col_name], id_df_unique.Stage):
          cv_tr = np.where(id_df.loc[:,id_col_name].isin(id_df_unique.iloc[train_idx,0]))[0]
          cv_val = np.where(id_df.loc[:,id_col_name].isin(id_df_unique.iloc[val_idx,0]))[0]
          sample_ids.append((cv_tr, cv_val))
      return(sample_ids)

In [12]:
from sklearn.metrics import confusion_matrix, roc_auc_score
def calc_metrics(y_true, y_pred):
    """Computes confusion matrix, accuracy, precision, recall, specificity, f1-score, and AUC for true and predicted labels

    Parameters
    ----------
    y_true : numpy.array
        The true labels
    y_pred : numpy.array
        The predicted label

    Returns
    -------
    tuple
        The first element is a confusion matrix, the second is a dictionary of remaining metrics
    """
    cm = confusion_matrix(np.ravel(y_true), y_pred.astype('int32'))
    acc = np.mean(y_true == y_pred.astype('int32'))
    auc = roc_auc_score(np.ravel(y_true), y_pred.astype('int32'))
    tn = cm[0,0]
    fn = cm[1,0]
    tp = cm[1,1]
    fp = cm[0,1]
    
    if(tp + fp == 0):
        precision = np.nan
    else:
        precision = tp/(tp + fp)
        
    if(tp + fn == 0):
        recall = np.nan
    else:
        recall = tp/(tp + fn)
    
    if(tn + fp == 0):
        specificity = np.nan
    else:
        specificity = tn/(tn + fp)
        
    if(precision + recall == 0):
        f1 = np.nan
    else:
        f1 = 2 * ((precision * recall)/(precision + recall))
    
    return((cm, {'Accuracy': [acc], 'AUC': [auc], 'Precision': [precision], 'Recall': [recall], 'Specificity': [specificity], 'F1-score': [f1]}))

In [ ]:
# Voting Classifier Ensemble 
seed = 123
multi_accs = []
losses = []
multi_accs_m = []
cms = {'0': [], '1': [], '2': []}
cms_m = {'0': [], '1': [], '2': []}
metrics = {'0': [], '1': [], '2': []}
metrics_m = {'0': [], '1': [], '2': []}
sub_accs = {'resnet': [], 'densenet': [], 'inception': [], 'vgg': []}
sub_accs_m = {'resnet': [], 'densenet': [], 'inception': [], 'vgg': []}
sub_losses = {'resnet': [], 'densenet': [], 'inception': [], 'vgg': []}

id_df = te.iloc[:,1:]
id_df_unique = id_df.drop_duplicates('Mouse_ID')
y_m = id_df_unique.Stage.values

for i in range(30):
    print("==========================================")
    print('Iteration', str(i))

    #Resnet
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    kind = 'resnet'
    testmod_resnet = build_cnn_base(1, kind = kind, nlayer = 20, 
                            optimizer = optimizers.SGD(learning_rate = 0.00001), 
                            optimizer2 = optimizers.Adamax(learning_rate = 0.005), input_shape = (min_dim0, min_dim1, 3), random_state = seed)
    testmod_resnet.fit(np.array([img for img in tr.Image.values]), to_categorical(tr.Stage.values), 
                epochs = 3,
                batch_size = 3
                )
    res_eval = testmod_resnet.evaluate(np.array([img for img in te.Image.values]), to_categorical(te.Stage.values))
    sub_accs['resnet'].append(res_eval[1])
    sub_losses['resnet'].append(res_eval[0])
    preds_r = np.argmax(testmod_resnet.predict(np.array([img for img in te.Image.values])), axis = 1)
    id_df['preds'] = preds_r
    preds_mr = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    sub_accs_m['resnet'].append(np.mean(y_m == preds_mr))

    #Densenet
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    kind = 'densenet'
    testmod_densenet = build_cnn_base(3, kind = kind, nlayer = 5, 
                            optimizer = optimizers.SGD(learning_rate = 0.00001),
                            optimizer2 = optimizers.Nadam(learning_rate = 0.005), input_shape = (min_dim0, min_dim1, 3), random_state = seed)
    testmod_densenet.fit(np.array([img for img in tr.Image.values]), to_categorical(tr.Stage.values),  epochs = 15, 
                batch_size = 3
                )
    dense_eval = testmod_densenet.evaluate(np.array([img for img in te.Image.values]), to_categorical(te.Stage.values))
    sub_accs['densenet'].append(dense_eval[1])
    sub_losses['densenet'].append(dense_eval[0])
    preds_d = np.argmax(testmod_densenet.predict(np.array([img for img in te.Image.values])), axis = 1)
    id_df['preds'] = preds_d
    preds_md = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    sub_accs_m['densenet'].append(np.mean(y_m == preds_md))

    #VGG
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    kind = 'vgg'
    testmod_vgg = build_cnn_base(0, kind = kind, nlayer = 8, 
                            optimizer = optimizers.SGD(learning_rate = 0.0001),
                            optimizer2 = optimizers.Nadam(learning_rate = 0.001), input_shape = (min_dim0, min_dim1, 3), random_state = seed)
    testmod_vgg.fit(np.array([img for img in tr.Image.values]), to_categorical(tr.Stage.values),  epochs = 3, 
                batch_size = 3
                )
    vgg_eval = testmod_vgg.evaluate(np.array([img for img in te.Image.values]), to_categorical(te.Stage.values))
    sub_accs['vgg'].append(vgg_eval[1])
    sub_losses['vgg'].append(vgg_eval[0])
    preds_v = np.argmax(testmod_vgg.predict(np.array([img for img in te.Image.values])), axis = 1)
    id_df['preds'] = preds_v
    preds_mv = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    sub_accs_m['vgg'].append(np.mean(y_m == preds_mv))

    #Inception
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    kind = 'inception'
    testmod_inception = build_cnn_base(2, kind = kind, nlayer = 30, 
                            optimizer = optimizers.SGD(learning_rate = 0.0001), 
                            optimizer2 = optimizers.Nadam(learning_rate = 0.005), input_shape = (min_dim0, min_dim1, 3), random_state = seed)
    testmod_inception.fit(np.array([img for img in tr.Image.values]), to_categorical(tr.Stage.values),  epochs = 10, 
                batch_size = 3
                )
    incept_eval = testmod_inception.evaluate(np.array([img for img in te.Image.values]), to_categorical(te.Stage.values))
    sub_accs['inception'].append(incept_eval[1])
    sub_losses['inception'].append(incept_eval[0])
    preds_i = np.argmax(testmod_inception.predict(np.array([img for img in te.Image.values])), axis = 1)
    id_df['preds'] = preds_i
    preds_mi = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    sub_accs_m['inception'].append(np.mean(y_m == preds_mi))

    #FCNet
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    #mega_fc = make_FCNet([testmod_vgg, testmod_resnet, testmod_inception, testmod_densenet], activation = PReLU(), n_output = 3, loss = 'categorical_crossentropy', optimizer = 'nadam',
     #                   clipvalue = 6, metrics = ['accuracy'], nhidden1 = 30, nhidden2 = 30, nhidden3 = 30,
      #                  dropout1 = 0, dropout2 = 0.1, dropout3 = 0, lr = 0.0001, seed = seed)
    #mega_fc.fit([np.array([img for img in tr.Image.values])] * 4, to_categorical(tr.Stage.values), batch_size = 3, epochs = 20)
    
    # Metrics
    # Per image
    #preds = np.argmax(mega_fc.predict([np.array([img for img in te.Image.values])] * 4), axis = 1)
    preds = np.max(pd.DataFrame(np.array([preds_r, preds_d, preds_v, preds_i])).mode(), axis = 0)
    preds_0 = np.array([1 if pred == 0 else 0 for pred in preds])
    y_0 = np.array([1 if i == 0 else 0 for i in te.Stage.values])
    preds_1 = np.array([1 if pred == 1 else 0 for pred in preds])
    y_1 = np.array([1 if i == 1 else 0 for i in te.Stage.values])
    preds_2 = np.array([1 if pred == 2 else 0 for pred in preds])
    y_2 = np.array([1 if i == 2 else 0 for i in te.Stage.values])

    #eval = mega_fc.evaluate([np.array([img for img in te.Image.values])] * 4, to_categorical(te.Stage.values))
    #multi_accs.append(eval[1])
    #losses.append(eval[0])
    multi_accs.append(np.mean(preds == te.Stage.values))
    print(np.mean(preds == te.Stage.values))
    cm_0, metrics_0 = calc_metrics(y_0, preds_0)
    cm_1, metrics_1 = calc_metrics(y_1, preds_1)
    cm_2, metrics_2 = calc_metrics(y_2, preds_2)
    cms['0'].append(cm_0)
    cms['1'].append(cm_1)
    cms['2'].append(cm_2)
    metrics['0'].append(metrics_0)
    metrics['1'].append(metrics_1)
    metrics['2'].append(metrics_2)

    # per animal
    id_df['preds'] = preds
    preds_m = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    preds_m0 = np.array([1 if pred == 0 else 0 for pred in preds_m])
    y_m0 = np.array([1 if i == 0 else 0 for i in y_m])
    preds_m1 = np.array([1 if pred == 1 else 0 for pred in preds_m])
    y_m1 = np.array([1 if i == 1 else 0 for i in y_m])
    preds_m2 = np.array([1 if pred == 2 else 0 for pred in preds_m])
    y_m2 = np.array([1 if i == 2 else 0 for i in y_m])

    multi_accs_m.append(np.mean(y_m == preds_m))
    cm_m0, metrics_m0 = calc_metrics(y_m0, preds_m0)
    cm_m1, metrics_m1 = calc_metrics(y_m1, preds_m1)
    cm_m2, metrics_m2 = calc_metrics(y_m2, preds_m2)
    cms_m['0'].append(cm_m0)
    cms_m['1'].append(cm_m1)
    cms_m['2'].append(cm_m2)
    metrics_m['0'].append(metrics_m0)
    metrics_m['1'].append(metrics_m1)
    metrics_m['2'].append(metrics_m2)

    seed += 1

print("==========================================")
print("Done!")

Iteration 0
Epoch 1/3
81/81 [==============================] - 8s 28ms/step - loss: 3.2076 - accuracy: 0.7769
Epoch 2/3
81/81 [==============================] - 2s 21ms/step - loss: 0.0936 - accuracy: 0.9835
Epoch 3/3
4/4 [==============================] - 1s 78ms/step
Epoch 1/15
81/81 [==============================] - 12s 32ms/step - loss: 11.0015 - accuracy: 0.7190
Epoch 2/15
81/81 [==============================] - 2s 20ms/step - loss: 1.4863 - accuracy: 0.9256
Epoch 3/15
81/81 [==============================] - 2s 20ms/step - loss: 0.9358 - accuracy: 0.9628
Epoch 4/15
81/81 [==============================] - 2s 20ms/step - loss: 0.5727 - accuracy: 0.9711
Epoch 5/15
81/81 [==============================] - 2s 20ms/step - loss: 0.5553 - accuracy: 0.9793
Epoch 6/15
81/81 [==============================] - 2s 19ms/step - loss: 0.3882 - accuracy: 0.9835
Epoch 7/15
81/81 [==============================] - 2s 19ms/step - loss: 0.3021 - accuracy: 0.9793
Epoch 8/15
81/81 [=================

In [ ]:
results = [multi_accs, losses, multi_accs_m, cms, cms_m, metrics, metrics_m, sub_accs, sub_accs_m, sub_losses]
#save_object(results, dir + 'results_1_1.pkl')

In [58]:
results = read_object(dir + 'results_1_1')

In [59]:
# Sub Model Scores
print("Submodel Patient-wise Accuracies:")
print(pd.DataFrame(results[-2]).agg([np.nanmean, np.nanstd]))

print()
print("Submodel Sample-wise Accuracies:")
print(pd.DataFrame(results[-3]).agg([np.nanmean, np.nanstd]))

print()
print("Submodel Sample-wise Losses:")
print(pd.DataFrame(results[-1]).agg([np.nanmean, np.nanstd]))

Submodel Patient-wise Accuracies:
           resnet  densenet  inception       vgg
nanmean  0.485185  0.400000   0.437037  0.455556
nanstd   0.094476  0.111494   0.091965  0.053407

Submodel Sample-wise Accuracies:
           resnet  densenet  inception       vgg
nanmean  0.629630  0.589562   0.602694  0.660943
nanstd   0.052206  0.086628   0.061547  0.061681

Submodel Sample-wise Losses:
           resnet   densenet  inception       vgg
nanmean  9.101143  74.950497  22.911459  1.099584
nanstd   1.944427  23.928473   6.687026  0.347265


In [60]:
# Voting Classifier Ensemble Scores
print("Ensemble Patient-wise Accuracy:")
print(pd.DataFrame({'Ensemble': results[2]}).agg([np.nanmean, np.nanstd]))

print()
print("Ensemble Sample-wise Accuracy:")
print(pd.DataFrame({'Ensemble': results[0]}).agg([np.nanmean, np.nanstd]))

for i in range(3):
    print()
    print("Ensemble Sample-wise Metrics: ", str(i), 'vs All')
    print(pd.DataFrame(results[5][str(i)]).apply(pd.Series.explode).agg([np.nanmean, np.nanstd]))

for i in range(3):
    print()
    print("Ensemble Patient-wise Metrics: ", str(i), 'vs All')
    print(pd.DataFrame(results[6][str(i)]).apply(pd.Series.explode).agg([np.nanmean, np.nanstd]))

Ensemble Patient-wise Accuracy:
         Ensemble
nanmean  0.462963
nanstd   0.051237

Ensemble Sample-wise Accuracy:
         Ensemble
nanmean  0.643771
nanstd   0.044934

Ensemble Sample-wise Metrics:  0 vs All
         Accuracy       AUC  Precision    Recall  Specificity  F1-score
nanmean  0.787205  0.536905   0.739524  0.086364     0.987446  0.173315
nanstd   0.014269  0.031066   0.238774  0.068935     0.016876  0.088379

Ensemble Sample-wise Metrics:  1 vs All
         Accuracy       AUC  Precision    Recall  Specificity  F1-score
nanmean  0.844781  0.773990   0.957410  0.561616     0.986364  0.699413
nanstd   0.039209  0.059521   0.050969  0.123472     0.019227  0.099414

Ensemble Sample-wise Metrics:  2 vs All
         Accuracy       AUC  Precision    Recall  Specificity  F1-score
nanmean  0.655556  0.688409   0.567121  0.984091     0.392727  0.718452
nanstd   0.045972  0.040745   0.037035  0.031099     0.091919  0.026572

Ensemble Patient-wise Metrics:  0 vs All
         Accura

In [ ]:
# FCNet Ensemble 
seed = 123
multi_accs = []
losses = []
multi_accs_m = []
cms = {'0': [], '1': [], '2': []}
cms_m = {'0': [], '1': [], '2': []}
metrics = {'0': [], '1': [], '2': []}
metrics_m = {'0': [], '1': [], '2': []}
sub_accs = {'resnet': [], 'densenet': [], 'inception': [], 'vgg': []}
sub_accs_m = {'resnet': [], 'densenet': [], 'inception': [], 'vgg': []}
sub_losses = {'resnet': [], 'densenet': [], 'inception': [], 'vgg': []}

id_df = te.iloc[:,1:]
id_df_unique = id_df.drop_duplicates('Mouse_ID')
y_m = id_df_unique.Stage.values

for i in range(30):
    print("==========================================")
    print('Iteration', str(i))

    #Resnet
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    kind = 'resnet'
    testmod_resnet = build_cnn_base(1, kind = kind, nlayer = 20, 
                            optimizer = optimizers.SGD(learning_rate = 0.00001), 
                            optimizer2 = optimizers.Adamax(learning_rate = 0.005), input_shape = (min_dim0, min_dim1, 3), random_state = seed)
    testmod_resnet.fit(X_tr, to_categorical(y_tr), 
                epochs = 3,
                batch_size = 3
                )
    res_eval = testmod_resnet.evaluate(np.array([img for img in te.Image.values]), to_categorical(te.Stage.values))
    sub_accs['resnet'].append(res_eval[1])
    sub_losses['resnet'].append(res_eval[0])
    preds_r = np.argmax(testmod_resnet.predict(np.array([img for img in te.Image.values])), axis = 1)
    id_df['preds'] = preds_r
    preds_mr = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    sub_accs_m['resnet'].append(np.array(y_m == preds_mr))

    #Densenet
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    kind = 'densenet'
    testmod_densenet = build_cnn_base(3, kind = kind, nlayer = 5, 
                            optimizer = optimizers.SGD(learning_rate = 0.00001),
                            optimizer2 = optimizers.Nadam(learning_rate = 0.005), input_shape = (min_dim0, min_dim1, 3), random_state = seed)
    testmod_densenet.fit(X_tr, to_categorical(y_tr),  epochs = 15, 
                batch_size = 3
                )
    dense_eval = testmod_densenet.evaluate(np.array([img for img in te.Image.values]), to_categorical(te.Stage.values))
    sub_accs['densenet'].append(dense_eval[1])
    sub_losses['densenet'].append(dense_eval[0])
    preds_d = np.argmax(testmod_densenet.predict(np.array([img for img in te.Image.values])), axis = 1)
    id_df['preds'] = preds_d
    preds_md = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    sub_accs_m['densenet'].append(np.array(y_m == preds_md))

    #VGG
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    kind = 'vgg'
    testmod_vgg = build_cnn_base(0, kind = kind, nlayer = 8, 
                            optimizer = optimizers.SGD(learning_rate = 0.0001),
                            optimizer2 = optimizers.Nadam(learning_rate = 0.001), input_shape = (min_dim0, min_dim1, 3), random_state = seed)
    testmod_vgg.fit(X_tr, to_categorical(y_tr),  epochs = 3, 
                batch_size = 3
                )
    vgg_eval = testmod_vgg.evaluate(np.array([img for img in te.Image.values]), to_categorical(te.Stage.values))
    sub_accs['vgg'].append(vgg_eval[1])
    sub_losses['vgg'].append(vgg_eval[0])
    preds_v = np.argmax(testmod_vgg.predict(np.array([img for img in te.Image.values])), axis = 1)
    id_df['preds'] = preds_v
    preds_mv = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    sub_accs_m['vgg'].append(np.array(y_m == preds_mv))

    #Inception
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    kind = 'inception'
    testmod_inception = build_cnn_base(2, kind = kind, nlayer = 30, 
                            optimizer = optimizers.SGD(learning_rate = 0.0001), 
                            optimizer2 = optimizers.Nadam(learning_rate = 0.005), input_shape = (min_dim0, min_dim1, 3), random_state = seed)
    testmod_inception.fit(X_tr, to_categorical(y_tr),  epochs = 10, 
                batch_size = 3
                )
    incept_eval = testmod_inception.evaluate(np.array([img for img in te.Image.values]), to_categorical(te.Stage.values))
    sub_accs['inception'].append(incept_eval[1])
    sub_losses['inception'].append(incept_eval[0])
    preds_i = np.argmax(testmod_inception.predict(np.array([img for img in te.Image.values])), axis = 1)
    id_df['preds'] = preds_i
    preds_mi = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    sub_accs_m['inception'].append(np.array(y_m == preds_mi))

    #FCNet
    K.clear_session()
    np.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    mega_fc = make_FCNet([testmod_vgg, testmod_resnet, testmod_inception, testmod_densenet], activation = PReLU(), n_output = 3, loss = 'categorical_crossentropy', optimizer = 'nadam',
                        clipvalue = 6, metrics = ['accuracy'], nhidden1 = 552, nhidden2 = 552, nhidden3 = 552,
                        dropout1 = 0.1, dropout2 = 0.1, dropout3 = 0.1, lr = 0.0001, seed = seed)
    mega_fc.fit([np.array([img for img in tr.Image.values])] * 4, to_categorical(tr.Stage.values), batch_size = 3, epochs = 10)
    
    # Metrics
    # Per image
    preds = np.argmax(mega_fc.predict([np.array([img for img in te.Image.values])] * 4), axis = 1)
    #preds = np.max(pd.DataFrame(np.array([preds_r, preds_d, preds_v, preds_i])).mode(), axis = 0)
    preds_0 = np.array([1 if pred == 0 else 0 for pred in preds])
    y_0 = np.array([1 if i == 0 else 0 for i in te.Stage.values])
    preds_1 = np.array([1 if pred == 1 else 0 for pred in preds])
    y_1 = np.array([1 if i == 1 else 0 for i in te.Stage.values])
    preds_2 = np.array([1 if pred == 2 else 0 for pred in preds])
    y_2 = np.array([1 if i == 2 else 0 for i in te.Stage.values])

    eval = mega_fc.evaluate([np.array([img for img in te.Image.values])] * 4, to_categorical(te.Stage.values))
    multi_accs.append(eval[1])
    losses.append(eval[0])
    #multi_accs.append(np.mean(preds == te.Stage.values))
    cm_0, metrics_0 = calc_metrics(y_0, preds_0)
    cm_1, metrics_1 = calc_metrics(y_1, preds_1)
    cm_2, metrics_2 = calc_metrics(y_2, preds_2)
    cms['0'].append(cm_0)
    cms['1'].append(cm_1)
    cms['2'].append(cm_2)
    metrics['0'].append(metrics_0)
    metrics['1'].append(metrics_1)
    metrics['2'].append(metrics_2)

    # per animal
    id_df['preds'] = preds
    preds_m = id_df.groupby(['Mouse_ID'])['preds'].agg(lambda x: pd.Series.mode(x)[0]).values
    preds_m0 = np.array([1 if pred == 0 else 0 for pred in preds_m])
    y_m0 = np.array([1 if i == 0 else 0 for i in y_m])
    preds_m1 = np.array([1 if pred == 1 else 0 for pred in preds_m])
    y_m1 = np.array([1 if i == 1 else 0 for i in y_m])
    preds_m2 = np.array([1 if pred == 2 else 0 for pred in preds_m])
    y_m2 = np.array([1 if i == 2 else 0 for i in y_m])

    multi_accs_m.append(np.mean(y_m == preds_m))
    cm_m0, metrics_m0 = calc_metrics(y_m0, preds_m0)
    cm_m1, metrics_m1 = calc_metrics(y_m1, preds_m1)
    cm_m2, metrics_m2 = calc_metrics(y_m2, preds_m2)
    cms_m['0'].append(cm_m0)
    cms_m['1'].append(cm_m1)
    cms_m['2'].append(cm_m2)
    metrics_m['0'].append(metrics_m0)
    metrics_m['1'].append(metrics_m1)
    metrics_m['2'].append(metrics_m2)

    seed += 1

print("==========================================")
print("Done!")


Iteration 0
Epoch 1/3
52/52 [==============================] - 6s 30ms/step - loss: 4.9004 - accuracy: 0.7727
Epoch 2/3
52/52 [==============================] - 1s 20ms/step - loss: 0.2743 - accuracy: 0.9740
Epoch 3/3
4/4 [==============================] - 1s 72ms/step
Epoch 1/15
52/52 [==============================] - 10s 36ms/step - loss: 11.4861 - accuracy: 0.6948
Epoch 2/15
52/52 [==============================] - 1s 19ms/step - loss: 2.3389 - accuracy: 0.9351
Epoch 3/15
52/52 [==============================] - 1s 19ms/step - loss: 0.1732 - accuracy: 0.9740
Epoch 4/15
52/52 [==============================] - 1s 18ms/step - loss: 1.0734 - accuracy: 0.9545
Epoch 5/15
52/52 [==============================] - 1s 18ms/step - loss: 0.7220 - accuracy: 0.9675
Epoch 6/15
52/52 [==============================] - 1s 19ms/step - loss: 1.5482e-09 - accuracy: 1.0000
Epoch 7/15
52/52 [==============================] - 1s 18ms/step - loss: 0.6063 - accuracy: 0.9805
Epoch 8/15
52/52 [=============

In [ ]:
results = [multi_accs, losses, multi_accs_m, cms, cms_m, metrics, metrics_m, sub_accs, sub_accs_m, sub_losses]
#save_object(results, dir + 'results_2.pkl')

In [61]:
results = read_object(dir + 'results_2')

In [63]:
# FCNet Ensemble Scores
print("FCNet Patient-wise Accuracy:")
print(pd.DataFrame({'Ensemble': results[2]}).agg([np.nanmean, np.nanstd]))

print()
print("FCNet Sample-wise Accuracy:")
print(pd.DataFrame({'Ensemble': results[0]}).agg([np.nanmean, np.nanstd]))

print()
print("FCNet Sample-wise Loss:")
print(pd.DataFrame({'Ensemble': results[1]}).agg([np.nanmean, np.nanstd]))

for i in range(3):
    print()
    print("FCNet Sample-wise Metrics: ", str(i), 'vs All')
    print(pd.DataFrame(results[5][str(i)]).apply(pd.Series.explode).agg([np.nanmean, np.nanstd]))

for i in range(3):
    print()
    print("FCNet Patient-wise Metrics: ", str(i), 'vs All')
    print(pd.DataFrame(results[6][str(i)]).apply(pd.Series.explode).agg([np.nanmean, np.nanstd]))

FCNet Patient-wise Accuracy:
         Ensemble
nanmean  0.411111
nanstd   0.092962

FCNet Sample-wise Accuracy:
         Ensemble
nanmean  0.608081
nanstd   0.083523

FCNet Sample-wise Loss:
          Ensemble
nanmean  17.466053
nanstd    1.352158

FCNet Sample-wise Metrics:  0 vs All
         Accuracy       AUC  Precision    Recall  Specificity  F1-score
nanmean  0.721886  0.520346   0.275290  0.157576     0.883117  0.228737
nanstd   0.101070  0.037603   0.181068  0.188639     0.177611  0.086677

FCNet Sample-wise Metrics:  1 vs All
         Accuracy       AUC  Precision    Recall  Specificity  F1-score
nanmean  0.847138  0.787121   0.941702  0.607071     0.967172  0.728445
nanstd   0.054151  0.087455   0.099794  0.214641     0.075084  0.109676

FCNet Sample-wise Metrics:  2 vs All
         Accuracy       AUC  Precision    Recall  Specificity  F1-score
nanmean  0.647138  0.665833   0.579708  0.834091     0.497576  0.669200
nanstd   0.049622  0.048994   0.052961  0.171031     0.157747 